In [1]:
%pip install -q git+https://github.com/DOJO-Smart-Ways/DOJO-Beam-Transforms.git@main#egg=dojo-beam-transforms

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-api-python-client 1.8.0 requires google-api-core<2dev,>=1.13.0, but you have google-api-core 2.22.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [15]:
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions
from datetime import datetime
import os
from google.cloud import bigquery

# GCP Project
gcp_project_id = "amr-east-lakehouse"
os.environ["GOOGLE_CLOUD_PROJECT"] = gcp_project_id
print(gcp_project_id)

amr-east-lakehouse


In [16]:
from pipeline_components import input_file as io
from utils.gcp_utils import build_gcs_path, build_bq_schema

In [ ]:
# Config Pipeline Options and Create a new Pipeline
# pipeline_options = {'project': 'us-multi'}
template_id = "CUST_MAST_STEPH_material_to_refined"
project = "Portfolio_Harmonization_Phase_1"
# Define runner type
runner_type = 'DataFlowRunner' # use 'DataFlowRunner' for creating a template for Dataflow.

pipeline_options = {
    "project": gcp_project_id,
    "runner": runner_type,
    "region": "us-central1",
    "staging_location": build_gcs_path(
        "thotasales_bucket", "data_flow_pipelines", project, "staging"
    ),
    "temp_location": build_gcs_path(
        "thotasales_bucket", "data_flow_pipelines", project, "temp"
    ),
    "template_location": build_gcs_path(
        "thotasales_bucket", "data_flow_pipelines", project, "template", template_id
    )
}

pipeline_options = PipelineOptions.from_dictionary(pipeline_options)
pipeline = beam.Pipeline(options=pipeline_options)

In [ ]:
# def build_raw_material_schema():
#     schema = [
#         ("customer_number", "STRING"),
#         ("customer_name", "STRING"),
#         ("cust_name2", "STRING"),
#         ("alternate_name", "STRING"),
#         ("partner_type", "STRING"),
#         ("partner_cust_no", "STRING"),
#         ("duns", "STRING"),
#         ("sic_code", "STRING"),
#         ("comp_code", "STRING"),
#         ("del_flag", "STRING"),
#         ("record_creation", "STRING"),
#         ("change_date", "STRING"),
#         ("partner_counter", "STRING"),
#         ("primary_bill_to", "STRING"),
#         ("street", "STRING"),
#         ("street2", "STRING"),
#         ("street3", "STRING"),
#         ("rg", "STRING"),
#         ("credit_limit", "STRING"),
#         ("currency_code", "STRING"),
#         ("primary_salesrep", "STRING"),
#         ("market_class", "STRING"),
#         ("cty", "STRING"),
#         ("country_name", "STRING"),
#         ("dist_channel", "STRING"),
#         ("sales_org", "STRING"),
#         ("pay_terms", "STRING"),
#         ("name", "STRING"),
#         ("SHORT_CODE", "STRING"),
#         ("UPDATE_DATE", "DATE")
#     ]
#     return build_bq_schema(schema)

In [ ]:
# 'Raw Material' Query
# Dummy function 'get_rm_result_query' for query generation (replace with your actual function)
def get_rm_result_query(table_name):
    return f"""
WITH Watermark AS (
  SELECT 
    IF(watermark_date IS NULL, DATE '1900-01-01', watermark_date) AS watermark_date
  FROM (
    SELECT 
      MAX(DATE_SUB(UPDATE_DATE, INTERVAL 2 DAY)) AS watermark_date
    FROM 
      `{gcp_project_id}.refined.{table_name}`
  )
)
SELECT
    cust.customer_number,
    cust.customer_name,
    cust.cust_name2,
    cust.alternate_name,
    cust.partner_type,
    cust.partner_cust_no,
    cust.duns,
    cust.sic_code,
    cust.comp_code,
    cust.del_flag,
    cust.record_creation,
    cust.change_date,
    cust.partner_counter,
    cust.primary_bill_to,
    cust.street,
    cust.street2,
    cust.street3,
    cust.rg,
    cust.credit_limit,
    cust.currency_code,
    cust.primary_salesrep,
    cust.market_class,
    cust.cty,
    cust.country_name,
    cust.dist_channel,
    cust.sales_org,
    cust.pay_terms,
    cust.name,
    cust.SHORT_CODE,
    cust.UPDATE_DATE
FROM (
  SELECT DISTINCT
    hca.account_number AS customer_number,
    hp.party_name AS customer_name,
    REGEXP_REPLACE(hca.account_name, r'\s+', ' ') AS cust_name2,
    hp.known_as2 AS alternate_name,
    hp.party_type AS partner_type,
    hp.party_number AS partner_cust_no,
    hp.duns_number AS duns,
    hp.sic_code AS sic_code,
    CASE 
      WHEN hou.SHORT_CODE = 'LEX' THEN '4001'
      WHEN hou.SHORT_CODE = 'KTO' THEN '4000'
      WHEN hou.SHORT_CODE = 'LPD' THEN '4002'
      ELSE gl.segment1
    END AS comp_code,
    CASE 
      WHEN hca.status = 'I' THEN 'Y' 
      ELSE 'N' 
    END AS del_flag,
    FORMAT_DATE('%d-%b-%Y', hca.creation_date) AS record_creation,
    FORMAT_DATE('%d-%b-%Y', hca.last_update_date) AS change_date,
    hps.party_site_number AS partner_counter,
    hcasa.bill_to_flag AS primary_bill_to,
    hl.address1 AS street,
    hl.address2 AS street2,
    hl.address3 AS street3,
    hl.state AS rg,
    hcpa.trx_credit_limit AS credit_limit,
    hcpa.currency_code,
    jrs.salesrep_number AS primary_salesrep,
    hcsua.attribute1 AS market_class,
    hl.county AS cty,
    hl.country AS country_name,
    hca.sales_channel_code AS dist_channel,
    hou.name AS sales_org,
    rt.name AS pay_terms,
    hou.name,
    hou.SHORT_CODE,
    CURRENT_DATE() AS UPDATE_DATE
  FROM
    `{gcp_project_id}.transient.steph_hz_parties` hp
  JOIN 
    `{gcp_project_id}.transient.steph_hz_party_sites` hps 
      ON hp.party_id = hps.party_id
      AND (CAST(hp.last_update_date AS DATE) > (SELECT watermark_date FROM Watermark) 
      OR CAST(hps.last_update_date AS DATE) > (SELECT watermark_date FROM Watermark))
  JOIN 
    `{gcp_project_id}.transient.steph_hz_locations` hl 
      ON hl.location_id = hps.location_id
      AND CAST(hl.last_update_date AS DATE) > (SELECT watermark_date FROM Watermark)
  JOIN 
    `{gcp_project_id}.transient.steph_hz_cust_accounts` hca 
      ON hca.party_id = hp.party_id
      AND CAST(hca.last_update_date AS DATE) > (SELECT watermark_date FROM Watermark)
  JOIN 
    `{gcp_project_id}.transient.steph_hz_cust_acct_sites_all` hcasa 
      ON hca.cust_account_id = hcasa.cust_account_id
      AND CAST(hcasa.last_update_date AS DATE) > (SELECT watermark_date FROM Watermark)
  JOIN 
    `{gcp_project_id}.transient.steph_hz_cust_site_uses_all` hcsua 
      ON hcsua.cust_acct_site_id = hcasa.cust_acct_site_id 
      AND hcsua.site_use_code = 'BILL_TO'
      AND CAST(hcsua.last_update_date AS DATE) > (SELECT watermark_date FROM Watermark)
  JOIN 
    `{gcp_project_id}.transient.steph_gl_code_combinations` gl
      ON gl.code_combination_id = hcsua.gl_id_rev
      AND CAST(gl.last_update_date AS DATE) > (SELECT watermark_date FROM Watermark)
  JOIN 
    `{gcp_project_id}.transient.steph_hr_operating_units` hou 
      ON hou.organization_id = hcsua.org_id
      AND CAST(hou.last_update_date AS DATE) > (SELECT watermark_date FROM Watermark)
  JOIN 
    `{gcp_project_id}.transient.steph_ra_terms` rt 
      ON rt.organization_id = hcsua.org_id
      AND CAST(rt.last_update_date AS DATE) > (SELECT watermark_date FROM Watermark)
  LEFT JOIN 
    `{gcp_project_id}.transient.steph_jtf_rs_salesreps` jrs 
      ON hcsua.primary_salesrep_id = jrs.salesrep_id 
      AND hcsua.org_id = jrs.org_id
      AND CAST(jrs.last_update_date AS DATE) > (SELECT watermark_date FROM Watermark)
  LEFT JOIN (
    SELECT
      hr.subject_id,
      hpg.party_name
    FROM
      `{gcp_project_id}.transient.steph_hz_relationships` hr
    JOIN 
      `{gcp_project_id}.transient.steph_hz_parties` hpg 
        ON hpg.party_id = hr.object_id
        AND (CAST(hr.last_update_date AS DATE) > (SELECT watermark_date FROM Watermark) 
        OR CAST(hpg.last_update_date AS DATE) > (SELECT watermark_date FROM Watermark))
    WHERE
      hr.status = 'A'
      AND hr.object_table_name = 'HZ_PARTIES'
      AND hr.subject_table_name = 'HZ_PARTIES'
      AND hr.relationship_code IN ('MEMBER_OF', 'CHILD_OF')
  ) hpgrp 
    ON hp.party_id = hpgrp.subject_id
  LEFT JOIN (
    SELECT
      *
    FROM
      `{gcp_project_id}.transient.steph_hz_customer_profiles`
    WHERE
      site_use_id IS NULL
      AND CAST(last_update_date AS DATE) > (SELECT watermark_date FROM Watermark)
  ) hcp 
    ON rt.term_id = hcp.standard_terms
  LEFT JOIN (
    SELECT
      *
    FROM
      `{gcp_project_id}.transient.steph_hz_cust_profile_amts`
    WHERE
      currency_code = 'USD'
      AND CAST(last_update_date AS DATE) > (SELECT watermark_date FROM Watermark)
  ) hcpa 
    ON hca.cust_account_id = hcpa.cust_account_id
  LEFT JOIN (
    SELECT
      term_id,
      MAX(due_days) AS due_days
    FROM
      `{gcp_project_id}.transient.steph_ra_terms_lines`
    WHERE 
      CAST(last_update_date AS DATE) > (SELECT watermark_date FROM Watermark)
    GROUP BY term_id
  ) rtl 
    ON rt.term_id = rtl.term_id
  LEFT JOIN (
    SELECT
      acra.pay_from_customer,
      acra.org_id,
      acra.amount,
      FORMAT_DATE('%d-%b-%Y', acra.receipt_date) AS receipt_date
    FROM
      `{gcp_project_id}.transient.steph_ar_cash_receipts_all` acra
    WHERE
      acra.cash_receipt_id = (
        SELECT
          MAX(cash_receipt_id)
        FROM
          `{gcp_project_id}.transient.steph_ar_cash_receipts_all` acram
        WHERE
          acram.org_id = acra.org_id
          AND acram.pay_from_customer = acra.pay_from_customer
          AND CAST(last_update_date AS DATE) > (SELECT watermark_date FROM Watermark)
      )
      AND CAST(last_update_date AS DATE) > (SELECT watermark_date FROM Watermark)
  ) acra 
    ON hcasa.org_id = acra.org_id 
    AND hcasa.cust_account_id = acra.pay_from_customer
  WHERE
  cust.comp_code='4002' OR
    (cust.SHORT_CODE IN ('LEX', 'KTO', 'LPD'))
) cust
ORDER BY
  cust.customer_number,
  cust.partner_counter;

"""


In [ ]:

# Main function of the Pipeline
def run_pipeline(temp_location, output_table,table_name):
    # Get the queries
    query = get_rm_result_query(table_name)

    # Read from BigQuery
    read_rw = io.read_bq(
        pipeline,
        query,
        temp_location,
        identifier="CUST_MAST STEPH Phase 1"
    )
    
    
    
    # Write to BigQuery
    read_rw | "Write To BigQuery" >> beam.io.WriteToBigQuery(
        output_table,
        # schema=build_raw_material_schema(),
        create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED,
        write_disposition=beam.io.BigQueryDisposition.WRITE_APPEND,
        custom_gcs_temp_location = temp_location,
    )

    pipeline.run().wait_until_finish()

In [ ]:
if __name__ == "__main__":
    output_table = "amr-east-lakehouse.refined.hrm_steph_custmast"
    table_name = 'hrm_steph_custmast'
    temp_location = build_gcs_path("thotasales_bucket", "data_flow_pipelines", "Portfolio_Harmonization_Phase_1", "temp")
    run_pipeline(temp_location, output_table, table_name)